# Get from Address to Model

In [2]:
import urllib
import requests
import json

## Geo-Admin API
Access of the API

In [3]:
#Definition of parameters
text = input("Please enter an address: ") #Input of the address to search for
base_url="https://api3.geo.admin.ch/rest/services/api/SearchServer?"
parameters = {"searchText": text, "origins": "address", "type": "locations",}

#Request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

data = json.loads(r.content)
print(data)

{'results': [{'attrs': {'detail': 'widmergasse 19 8466 truellikon 40 truellikon ch zh', 'featureId': '12870_0', 'geom_quadindex': '012232210032122231000', 'geom_st_box2d': 'BOX(694347.29027415 276686.7814323578,694347.29027415 276686.7814323578)', 'label': 'Widmergasse 19 <b>8466 Trüllikon</b>', 'lat': 47.63404846191406, 'lon': 8.694016456604004, 'num': 19, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276686.78125, 'y': 694347.3125, 'zoomlevel': 10}, 'id': 7555, 'weight': 1}, {'attrs': {'detail': 'widmergasse 19.1 8466 truellikon 40 truellikon ch zh', 'featureId': '210226908_0', 'geom_quadindex': '012232210032122310321', 'geom_st_box2d': 'BOX(694360.3364026649 276691.3415635696,694360.3364026649 276691.3415635696)', 'label': 'Widmergasse 19.1 <b>8466 Trüllikon</b>', 'lat': 47.634090423583984, 'lon': 8.694190979003906, 'num': 191, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276691.34375, 'y': 694360.3125, 'zoomlevel': 10}, 'id': 2195186, 'weight': 7}, {'attrs': {'

Geographic display of returned results

In [4]:
import folium

# Create a map centered at the first item's latitude and longitude
map_center = [data['results'][0]['attrs']['lat'], data['results'][0]['attrs']['lon']]
m = folium.Map(location=map_center, zoom_start=20)

# Add markers for each item with annotations
for i, result in enumerate(data['results']):
    lat = result['attrs']['lat']
    lon = result['attrs']['lon']
    label = f"Index: {i}"
    folium.Marker([lat, lon], popup=label).add_to(m)

# Display the map
m


In [5]:
#Select the desired item
index = int(input("Please enter the index of the item you want to select: "))
selected_item = data['results'][index]
print(selected_item)


{'attrs': {'detail': 'widmergasse 19 8466 truellikon 40 truellikon ch zh', 'featureId': '12870_0', 'geom_quadindex': '012232210032122231000', 'geom_st_box2d': 'BOX(694347.29027415 276686.7814323578,694347.29027415 276686.7814323578)', 'label': 'Widmergasse 19 <b>8466 Trüllikon</b>', 'lat': 47.63404846191406, 'lon': 8.694016456604004, 'num': 19, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276686.78125, 'y': 694347.3125, 'zoomlevel': 10}, 'id': 7555, 'weight': 1}


## KML files
Check KML files for desired building

In [8]:
import os
import re
import math

# Define the folder path
folder_path = "swissBUILDINGS3D 2.0"

# Get the latitude and longitude from the selected_item
lat = float(selected_item['attrs']['lat'])
lon = float(selected_item['attrs']['lon'])
print(f"Latitude: {lat}", f"Longitude: {lon}")

# Define the calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth radius in kilometers

    return distance

# Search for KML files in the folder
kml_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".kml"):
            kml_files.append(os.path.join(root, file))

print(f"Found {len(kml_files)} KML files", kml_files)

# Define the tolerance in kilometers
tolerance = 0.01

# List to store the filtered KML data
filtered_kml_data = []

# Iterate through the KML files
for kml_file in kml_files:
    with open(kml_file, 'r') as file:
        kml_content = file.read()
        
        # Extract the latitude and longitude values from the KML content
        latitudes = re.findall(r'<latitude>(.*?)</latitude>', kml_content)
        longitudes = re.findall(r'<longitude>(.*?)</longitude>', kml_content)
        hrefs = re.findall(r'<href>(.*?)</href>', kml_content)
        
        # Convert the latitude and longitude values to float
        latitudes = [float(lat) for lat in latitudes]
        longitudes = [float(lon) for lon in longitudes]
        
        # Check if any entry is within the tolerance
        for i in range(len(latitudes)):
            distance = calculate_distance(lat, lon, latitudes[i], longitudes[i])
            if distance <= tolerance:
                filtered_kml_data.append((kml_file, latitudes[i], longitudes[i],  hrefs[i]))

# Print the filtered KML data
for data in filtered_kml_data:
    print(f"KML File: {data[0]}, Latitude: {data[1]}, Longitude: {data[2]}, href: {data[3]}")



Latitude: 47.63404846191406 Longitude: 8.694016456604004
Found 6 KML files ['swissBUILDINGS3D 2.0\\1031-44\\1031-44.kml', 'swissBUILDINGS3D 2.0\\1032-33\\1032-33.kml', 'swissBUILDINGS3D 2.0\\1032-34\\1032-34.kml', 'swissBUILDINGS3D 2.0\\1051-22\\1051-22.kml', 'swissBUILDINGS3D 2.0\\1052-11\\1052-11.kml', 'swissBUILDINGS3D 2.0\\1052-12\\1052-12.kml']
KML File: swissBUILDINGS3D 2.0\1052-11\1052-11.kml, Latitude: 47.634047498746725, Longitude: 8.694078211780825, href: models/model_2045.dae
